In [2]:
! pip install -r requirements.txt --quiet

# Semenatic Kernel and the OpenAPI Plugin


🔗 [Add plugins from OpenAPI specifications](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/adding-openapi-plugins)

In [8]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.openapi_plugin import OpenAPIFunctionExecutionParameters
from dotenv import load_dotenv
from os import environ
import asyncio
import requests

load_dotenv(override=True)


True

In [9]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    service_id="chat",
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    base_url=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ))

kernel.add_plugin_from_openapi(
   plugin_name="sales",
   openapi_document_path=environ['OPENAPI_URL'],
   execution_settings=OpenAPIFunctionExecutionParameters(
         enable_payload_namespacing=True,
   ),
)

KernelPlugin(name='sales', description=None, functions={'list_customers_customers_customers_get': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='list_customers_customers_customers_get', plugin_name='sales', description='Retrieve customers', parameters=[KernelParameterMetadata(name='industry', description='Filter by industry', default_value='', type_='string', is_required=False, type_object=None, schema_data={'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'Filter by industry', 'title': 'Industry'}, include_in_function_choices=True), KernelParameterMetadata(name='account_manager', description='Filter by account manager', default_value='', type_='string', is_required=False, type_object=None, schema_data={'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'Filter by account manager', 'title': 'Account Manager'}, include_in_function_choices=True), KernelParameterMetadata(name='limit', description='Maximum number of customers to return', default_

In [ ]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the MCP tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.

Example user questions:
- "Which products generated the most revenue in the EU last quarter?"
- "Who are the top 5 customers by total sales this month?"
- "How much discount was given per product category in Q2?"
- "List all orders for customer 42 containing product 7."
"""

In [11]:
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgentOpenAPI",
    instructions=instructions
)

response = await agent.get_response(messages="Which products generated the most revenue?")
print(response)


Based on the sales data, the products that generated the highest revenue are:

1. **Brake Pad** - Total Revenue: **$29,230.00**
   - High sales volume and the relatively higher unit price ($50.00) contributed to this product's leading revenue.

2. **Oil Filter** - Total Revenue: **$17,730.00**
   - Large quantities sold ($15.00/unit price) ensured this product's significant revenue contribution.

3. **Spark Plug** - Total Revenue: **$11,690.00**
   - Though the unit price is low ($8.00), volume sales pushed its revenue.

4. **Alternator** and **Transmission Kit**
   - These products did not contribute significantly in revenue compared to others listed above, likely linked to sales volume rather than pricing.

Would you like a more detailed breakdown by region, time period, or other segmentation?
